In [1]:
import os

# Fill the environmental variable with your own OpenAI API key
# See: https://platform.openai.com/account/api-keys
os.environ["OPENAI_API_KEY"] = "x"

In [2]:
import qdrant_client

client = qdrant_client.QdrantClient("http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[])

In [3]:
COLLECTION_NAME="repo-embeddings"

In [4]:
from openai import OpenAI


In [5]:
oaclient = OpenAI()

In [6]:
import asyncio
from tqdm.asyncio import tqdm, trange
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, Payload, Filter
import time
import uuid

def get_embeddings(text, model="text-embedding-ada-002", encoding_format="float"):
   text = text.replace("\n", " ")
   return oaclient.embeddings.create(input=[text], model=model, encoding_format=encoding_format).data

In [7]:
TOP_N_RESULTS = 300
SCORE_THRESHOLD = 0.700
MAX_CONTEXT_CHARS = 10000
MAX_RESULT_CHARS =  5000
def rag(question: str, n_points: int = 100) -> str:
    embeddings = get_embeddings(question)

    results = []
    for embedding in embeddings:
        embedded_query = embedding.embedding

        embedding_results = client.search(
            collection_name=COLLECTION_NAME,
            query_vector=(
                "content", embedded_query
            ),
            limit=n_points,
        )
        results += embedding_results

    results = [result for result in results if round(result.score, 3) >= SCORE_THRESHOLD]
    results = sorted(results, key=lambda r: r.score, reverse=True)[:TOP_N_RESULTS]


    context = "\n".join(str(r.payload.get("output", ""))[:MAX_RESULT_CHARS] for r in results)[:MAX_CONTEXT_CHARS]

    metaprompt = f"""
    You are a software architect. 
    Answer the following question using the provided context. 
    If you can't find the answer, do not pretend you know it, but answer "I don't know".
    
    Question: {question.strip()}
    
    Context: you have the following list of Git Commits: {context.strip()}.
    
    Answer:
    """
    completion = oaclient.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": metaprompt},
        ],
        timeout=10.0,
    )
    output = completion.choices[0].message.content
    print(output.ljust(80, "\n"))

In [8]:
rag("Given the context you have, give me some insights regarding the commits with high bug_spot_likelyhood.", n_points=1000)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Not found: Collection `repo-embeddings` doesn't exist!"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-01-11T22:25:44.393988346+00:00", grpc_status:5, grpc_message:"Not found: Collection `repo-embeddings` doesn\'t exist!"}"
>